In [13]:
import numpy as np
import pandas as pd
artist_conv ={}
with open("artist_conversions.csv", 'r') as f:
    for line in f:
        l = line.split(',')
        a_id = l[0]
        artist_name = l[1]
        artist_name = artist_name.strip()
        artist_conv[a_id]  = artist_name

In [15]:
from surprise import Dataset
from surprise import Reader

reader = Reader(sep=',', rating_scale=(0,100))
data = Dataset.load_from_file( 'finalUserData.csv', reader=reader)
data.raw_ratings[:5]


[('1', '1', 1.0, None),
 ('2', '1', 61.0, None),
 ('3', '1', 1.0, None),
 ('4', '1', 1.0, None),
 ('5', '1', 101.0, None)]

In [ ]:
from surprise import SVD
from surprise.model_selection import train_test_split

trainset, testset = train_test_split( data , test_size= .50 , random_state=1)

algo = SVD( n_factors=10, n_epochs=500, random_state=1 )
algo.fit(trainset)

In [4]:
from surprise import accuracy
pred = algo.test(testset) 
accuracy.rmse(pred), accuracy.mae(pred)
error = "SVD: " + str(accuracy.rmse(pred))+"\n"
with open("RMSE.txt", "a") as w:
    w.write(error)

RMSE: 85.2499
MAE:  78.1681
RMSE: 85.2499


In [5]:

from collections import defaultdict
def get_top_n(predictions, n=3):
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n



top_n = get_top_n(pred, n=3)
count = 0
recommendations = []
for uid, user_ratings in top_n.items():
    count+=1
    #print(uid, [iid for (iid, _) in user_ratings])
    ur = [iid for (iid, _) in user_ratings]
    main = str(uid)+","+str(artist_conv[str(ur[0])])+ ","+str(artist_conv[str(ur[1])])+","+str(artist_conv[str(ur[2])])+"\n"

    print(main)
    recommendations.append(main)
print(count)

415,230,175,209

577,478,286,236

277,451,407,122

61,215,200,69

194,442,433,422

198,163,151,35

428,73,26,344

299,11,347,111

539,3,224,461

112,406,428,298

174,358,214,243

53,8,240,160

127,354,276,231

161,304,319,239

510,64,231,324

460,223,362,39

232,140,116,314

23,439,187,308

512,354,241,135

22,8,387,441

575,394,411,182

342,155,78,102

178,341,171,30

117,103,423,155

199,184,219,276

601,354,73,436

67,123,264,305

168,383,321,323

129,301,363,74

360,290,62,101

85,437,399,261

46,16,95,352

222,390,75,337

148,114,193,354

552,139,166,481

455,171,334,323

487,465,476,406

151,394,467,33

167,288,75,114

351,95,190,415

378,290,1,416

582,384,284,39

550,325,118,100

159,275,370,328

418,372,117,264

172,290,377,47

120,294,192,193

280,149,46,181

555,355,178,268

531,174,224,58

245,30,45,437

210,46,403,244

303,409,118,410

548,294,220,317

26,171,27,196

572,267,274,212

525,86,1,361

542,416,135,261

583,5,380,445

154,293,420,197

374,334,56,163

105,295,365

In [6]:
with open("SVD.txt", "w") as w:
    for x in recommendations:
        w.write(x)